# Aircraft Risk Analysis Project - Phase 1

## Introduction


In today's fast-evolving business landscape, diversification is key to staying competitive. This project explores a scenario where a company is looking to expand into the aviation industry—an exciting but high-risk move. While the aviation sector holds strong potential for both commercial and private enterprise, safety remains a top concern, especially for new entrants with limited industry experience.

The goal of this project is to use data science techniques to help the company make informed, data-driven decisions as it considers which aircraft to invest in. By examining historical accident data, we can uncover patterns, evaluate safety records, and highlight aircraft models that pose the least operational risk.

This project serves as both a business case and a demonstration of key data science skills: data cleaning, missing value imputation, exploratory data analysis, and visualization. The final output will include actionable insights and an interactive dashboard designed to support decision-making by the company’s new aviation division.

## Objectives

- Understand trends in aircraft accidents over time  
- Identify aircraft with the lowest risk profiles  
- Handle missing data and perform necessary cleaning  
- Present insights using clear visualizations  
- Build an interactive dashboard for stakeholders

## Data Source

The dataset comes from the National Transportation Safety Board (NTSB) and includes records of civil aviation accidents and selected incidents in the United States and international waters from 1962 to 2023.

## Tools & Libraries

- Python (Pandas, NumPy, Matplotlib, Seaborn)
- Jupyter Notebook
- Data cleaning and wrangling
- Exploratory data analysis (EDA)

### Step 1 : Import Libraries

In [1]:
#Importing libraries using standard alias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Step 2 : Load and Inspect Data

In [2]:
#Loading the csv file into the notebook
#Adding low_memory=False to allow pandas to read the full file before deciding datatypes
#Telling pandas to treat ? , Unknown , N/A and blank spaces as missing values
data= pd.read_csv('data/Aviation_Data.csv', low_memory=False, na_values=['?', 'Unknown', 'N/A', ''])

#Inspect the first 5 rows of the dataset
data.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,NaN,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [3]:
# Checking the number of rows and columns in the dataset
data.shape

# Displaying the dataset's dimensions (rows, columns)
print(f"The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.")

The dataset contains 90348 rows and 31 columns.


In [4]:
# Getting a summary of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88836 non-null  object 
 5   Country                 88660 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52783 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85576 non-null  object 
 12  Aircraft.Category       32273 non-null  object 
 13  Registration.Number     87569 non-null  object 
 14  Make                    88805 non-null

In [5]:
#Looking at all columns present
data.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [6]:
#Getting a summary statistics for numerical columns in the dataset
data.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


### Step 3 : Data Cleaning

In [7]:
# Creating a copy of the cleaned DataFrame to avoid modifying the original
cleaned_data = data.copy()

In [8]:
#Inspecting cleaned data
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88836 non-null  object 
 5   Country                 88660 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52783 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85576 non-null  object 
 12  Aircraft.Category       32273 non-null  object 
 13  Registration.Number     87569 non-null  object 
 14  Make                    88805 non-null

In [9]:
#Changing the Event.Date and Publication.Date columns into datetime
cleaned_data['Event.Date']=pd.to_datetime(cleaned_data['Event.Date'])
cleaned_data['Publication.Date']=pd.to_datetime(cleaned_data['Publication.Date'])

#Checking if it has been applied
cleaned_data[['Event.Date','Publication.Date']].head()

,Event.Date,Publication.Date
0,1948-10-24,NaT
1,1962-07-19,1996-09-19
2,1974-08-30,2007-02-26
3,1977-06-19,2000-12-09
4,1979-08-02,1980-04-16


In [10]:
# Checking if there are any duplicate rows
cleaned_data.duplicated().any()

True

In [11]:
# Droping duplicated rows and keeping the first occurrence
cleaned_data = cleaned_data.drop_duplicates()

#Checking dimensions of our cleaned data after duplicates have been dropped
cleaned_data.shape
print(f"The Cleaned dataset contains {cleaned_data.shape[0]} rows and {cleaned_data.shape[1]} columns.")

The Cleaned dataset contains 88958 rows and 31 columns.


In [12]:
#Grouping the numerical values and fill them with the mean
numeric_values = cleaned_data.select_dtypes(include=[float, int]).columns
cleaned_data[numeric_values] = cleaned_data[numeric_values].fillna(cleaned_data[numeric_values].mean())

In [13]:
#Checking for any null values in the numerical values
cleaned_data[numeric_values].isna().sum()

Number.of.Engines         0
Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

In [ ]:
# Selecting all object and datetime columns
categorical_values = cleaned_data.select_dtypes(include=['object', 'datetime64[ns]']).columns

# Looping through each column and filling missing values with the mode
for column in categorical_values:
    mode_series = cleaned_data[column].mode()

    # Only fills if the mode exists
    if not mode_series.empty:
        mode_value = mode_series.iloc[0]
        cleaned_data[column] = cleaned_data[column].fillna(mode_value)


In [17]:
#Check for any null values in the categoriacal values
cleaned_data[categorical_values].isna().sum()

Event.Id                 0
Investigation.Type       0
Accident.Number          0
Event.Date               0
Location                 0
Country                  0
Latitude                 0
Longitude                0
Airport.Code             0
Airport.Name             0
Injury.Severity          0
Aircraft.damage          0
Aircraft.Category        0
Registration.Number      0
Make                     0
Model                    0
Amateur.Built            0
Engine.Type              0
FAR.Description          0
Schedule                 0
Purpose.of.flight        0
Air.carrier              0
Weather.Condition        0
Broad.phase.of.flight    0
Report.Status            0
Publication.Date         0
dtype: int64

In [36]:
# Looping through each categorical column and applying strip() for each column
for column in categorical_values:
    # Converting the column to string type first, then applying strip() and making them lowercase
    cleaned_data[column] = cleaned_data[column].astype(str).str.strip().str.capitalize()

In [37]:
# Selecting numeric and categorical data by column names, then combining them back into one DataFrame
numerical_data = cleaned_data[numeric_values]
categorical_data = cleaned_data[categorical_values]
cleaned_data = pd.concat([numerical_data, categorical_data], axis=1)

#Checking the 1st 5 rows of the cleaned data
cleaned_data.head()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,...,Amateur.Built,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,1.000000,2.0,0.000000,0.000000,0.00000,20001218x45444,Accident,Sea87la080,1948-10-24,"Moose creek, id",...,No,Reciprocating,091,Nsch,Personal,Pilot,Unk,Cruise,Probable cause,2020-09-25
1,1.000000,4.0,0.000000,0.000000,0.00000,20001218x45447,Accident,Lax94la336,1962-07-19,"Bridgeport, ca",...,No,Reciprocating,091,Nsch,Personal,Pilot,Unk,Landing,Probable cause,1996-09-19
2,1.000000,3.0,0.279881,0.357061,5.32544,20061025x01555,Accident,Nyc07la005,1974-08-30,"Saltville, va",...,No,Reciprocating,091,Nsch,Personal,Pilot,Imc,Cruise,Probable cause,2007-02-26
3,1.000000,2.0,0.000000,0.000000,0.00000,20001218x45448,Accident,Lax96la321,1977-06-19,"Eureka, ca",...,No,Reciprocating,091,Nsch,Personal,Pilot,Imc,Cruise,Probable cause,2000-12-09
4,1.146585,1.0,2.000000,0.357061,0.00000,20041105x01764,Accident,Chi79fa064,1979-08-02,"Canton, oh",...,No,Reciprocating,091,Nsch,Personal,Pilot,Vmc,Approach,Probable cause,1980-04-16


In [38]:
#checking the data in the make column
cleaned_data['Make'].head(10)

0              Stinson
1                Piper
2               Cessna
3             Rockwell
4               Cessna
5    Mcdonnell douglas
6               Cessna
7               Cessna
8               Cessna
9       North american
Name: Make, dtype: object

In [39]:
#Inspecting information about the data after cleaning
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88958 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Number.of.Engines       88958 non-null  float64
 1   Total.Fatal.Injuries    88958 non-null  float64
 2   Total.Serious.Injuries  88958 non-null  float64
 3   Total.Minor.Injuries    88958 non-null  float64
 4   Total.Uninjured         88958 non-null  float64
 5   Event.Id                88958 non-null  object 
 6   Investigation.Type      88958 non-null  object 
 7   Accident.Number         88958 non-null  object 
 8   Event.Date              88958 non-null  object 
 9   Location                88958 non-null  object 
 10  Country                 88958 non-null  object 
 11  Latitude                88958 non-null  object 
 12  Longitude               88958 non-null  object 
 13  Airport.Code            88958 non-null  object 
 14  Airport.Name            88958 non-null

'''
The dataset has been thoroughly cleaned, with missing values handled. Duplicate rows have been removed, and data inside the columns cleaned. Following these data cleaning steps, the dataset now consists of 31 columns and 88,958 rows. The next step involves visualizing the cleaned data.
'''

### Step 4 : Data Analysis and Visualizations

In [40]:
#summary statistics for numerical columns in the cleaned dataset
cleaned_data.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,88958.000000,88958.000000,88958.000000,88958.000000,88958.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.430791,5.120083,1.430754,2.079349,26.958925
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.647855,0.279881,0.357061,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


'''
Key Analysis:

Fatal Injuries: Majority of the accidents habe no fatalities as seen in the 25%, 50% and 75% percentiles. However, there are extreme cases with upto 349 fatal injuries in one incident

Serious and Minor Injuries: Similar to the fatal injuries, majority of the incidents have no serious and minor injuries, but there are some outliers with other 161 serious injuries and 380 minor injuries

Uninjured individuals: Most accidesnts have some injured individals with a higher median value of 2 while in other cases many people were uninjured
'''

#### Step 4a : Accident Frequency by Aircraft Make

In [41]:
#Checking the columns i have
cleaned_data.columns

Index(['Number.of.Engines', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Event.Id',
       'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location',
       'Country', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name',
       'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category',
       'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Engine.Type',
       'FAR.Description', 'Schedule', 'Purpose.of.flight', 'Air.carrier',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [42]:
#Check if we can use the accident Number column for analysis
cleaned_data['Accident.Number'].head()

0    Sea87la080
1    Lax94la336
2    Nyc07la005
3    Lax96la321
4    Chi79fa064
Name: Accident.Number, dtype: object

Since the accident number column contains strings we can not use it for analysis. We will therefore create a new column called Total Accidents which will count the number of both minor and serious injuries

In [43]:
# Create a new column for Total Accidents (sum of Serious and Minor Injuries)
cleaned_data['Total.Accidents'] = cleaned_data['Total.Serious.Injuries'] + cleaned_data['Total.Minor.Injuries']

# Check if  the new column is added
cleaned_data[['Make', 'Model', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Accidents']].head()

,Make,Model,Total.Serious.Injuries,Total.Minor.Injuries,Total.Accidents
0,Stinson,108-3,0.000000,0.000000,0.000000
1,Piper,Pa24-180,0.000000,0.000000,0.000000
2,Cessna,172m,0.279881,0.357061,0.636942
3,Rockwell,112,0.000000,0.000000,0.000000
4,Cessna,501,2.000000,0.357061,2.357061


In [44]:
# Count the number of accidents per aircraft make
accidents_per_model = cleaned_data.groupby('Make').size().reset_index(name='Total.Accidents')

# Sort by the accident count for better readability
accidents_per_model = accidents_per_model.sort_values(by='Total.Accidents', ascending=False)

# Display the top 10 aircraft types with the most accidents
accidents_per_model.head(10)

,Make,Total.Accidents
1259,Cessna,27302
5276,Piper,14870
638,Beech,5372
827,Boeing,2745
658,Bell,2722
4753,Mooney,1334
5761,Robinson,1230
2853,Grumman,1172
688,Bellanca,1045
3352,Hughes,932
